# $H_{2}O$ continuum in RRTMG-LW

### First steps

In RRTMG-LW, four factors/terms are needed in order to calculate the absorption coefficient due to $H_2O$ continuum.  These, in the source code, are:

`forfac`  
`forfrac`  
`selffac`  
`selffrac`  

The '`for`' is for foreign-continuum, and the '`self`' is for self-continuum.

These four terms are calculated in 'rrtmg_lw_setcoef.f90'.

First, the variable `water` is calculated via:

```
water = wkl(1,lay)/coldry(lay)
```

`wkl`: molecule amount for water $[mol\: cm^{-2}]$  
`coldry`: dry air column density $[mol\: cm^{-2}]$

Then, `scalefac` is calculated via:

```
scalefac = pavel(lay) * stpfac / tavel(lay)
```

`stpfac`: $296 / 1013$  
`pavel`: layer pressure [mbar]  
`tavel`: layer temperature [K]   

Using `water` and `scalefac`, `forfac`, `forfrac`, `selffac` and `selffrac` can now be obtained.  But how this is obtained depends on whether the atmosphere layer is below or above the tropopause ($\sim 100$  mbar).

Below tropopause, `forfac`, `forfrac`, `selffac` and `selffrac` are obtained via:

```
         forfac(lay) = scalefac / (1.+water)
         factor = (332.0_rb-tavel(lay))/36.0_rb
         indfor(lay) = min(2, max(1, int(factor)))
         forfrac(lay) = factor - real(indfor(lay))
         
         
!  Set up factors needed to separately include the water vapor                                                          
!  self-continuum in the calculation of absorption coefficient.                                                         
         selffac(lay) = water * forfac(lay)
         factor = (tavel(lay)-188.0_rb)/7.2_rb
         indself(lay) = min(9, max(1, int(factor)-7))
         selffrac(lay) = factor - real(indself(lay) + 7)
```

Above tropopause, `forfac`, `forfrac` and `selffac` are obtained via (note there is no `selffrac`):

```
         forfac(lay) = scalefac / (1.+water)
         factor = (tavel(lay)-188.0_rb)/36.0_rb
         indfor(lay) = 3
         forfrac(lay) = factor - 1.0_rb

!  Set up factors needed to separately include the water vapor                                                          
!  self-continuum in the calculation of absorption coefficient.                                                         
         selffac(lay) = water * forfac(lay)
```